In [1]:
import tensorflow as tf

In [2]:
tf.io.decode_json_example(
    json_examples, name=None
)

NameError: name 'json_examples' is not defined

In [3]:
example = tf.train.Example(
  features=tf.train.Features(
      feature={
          "a": tf.train.Feature(
              int64_list=tf.train.Int64List(
                  value=[1, 1, 3]))}))

In [4]:
from google.protobuf import json_format
example_json = json_format.MessageToJson(example)
print(example_json)


{
  "features": {
    "feature": {
      "a": {
        "int64List": {
          "value": [
            "1",
            "1",
            "3"
          ]
        }
      }
    }
  }
}


In [7]:
example_binary = tf.io.decode_json_example(
    example_json, name=None
)

In [8]:
tf.io.parse_example(
  serialized=[example_binary.numpy(),
             example.SerializeToString()],
  features = {'a': tf.io.FixedLenFeature(shape=[3], dtype=tf.int64)})

{'a': <tf.Tensor: shape=(2, 3), dtype=int64, numpy=
 array([[1, 1, 3],
        [1, 1, 3]], dtype=int64)>}

In [9]:
import pandas as pd


In [13]:
recipes = pd.read_csv("../mock_data/mock-data_recipe_10.csv")

In [22]:
for row in range(len(recipes)):
    print(recipes[i])

NameError: name 'i' is not defined

In [18]:
recipes[:1].to_json("../mock_data/mock-data_recipe_10.json")

In [19]:
example = tf.io.read_file("../mock_data/mock-data_recipe_10.json")

In [20]:
tf.io.decode_json_example(example)

InvalidArgumentError: Error while parsing JSON: reviews: Cannot find field. [Op:DecodeJSONExample]

In [26]:
item_id = 55555

In [38]:
def create_tfrecord_for_item(pandas_line):
    record_filename = f'item_{item_id}.tfrecord'
    with tf.io.TFRecordWriter(record_filename) as writer:
   
            feature = {
                'recipe_id': tf.train.Feature(int64_list=tf.train.Int64List(value=pandas_line['recipe_id'].values)),
                # 'recipe_name': tf.train.Feature(int64_list=tf.train.Int64List(value=[pandas_line['recipe_name'].values()])),
                # 'rating': tf.train.Feature(float_list=tf.train.FloatList(value=[pandas_line['rating']])),
                # 'image_filepath': tf.train.Feature(bytes_list=tf.train.BytesList(value=[tf.io.encode_jpeg(tf.io.read_file(image_filepath)).numpy()])),
                # 'description': tf.train.Feature(bytes_list=tf.train.BytesList(value=[tf.io.encode_text("blabla")])),
            }
            example = tf.train.Example(features=tf.train.Features(feature=feature))
            writer.write(example.SerializeToString())
    return record_filename

In [39]:
recipes[:1]

,Unnamed: 0,recipe_id,recipe_name,aver_rate,image_url,review_nums,ingredients,cooking_directions,nutritions,reviews
0,0,222388,Homemade Bacon,5.0,https://images.media-allrecipes.com/userphotos...,3,pork belly^smoked paprika^kosher salt^ground b...,{'directions': u'Prep\n5 m\nCook\n2 h 45 m\nRe...,"{u'niacin': {u'hasCompleteData': False, u'name...","{8542392: {'rating': 5, 'followersCount': 11, ..."


In [40]:
create_tfrecord_for_item(recipes[:1])

'item_55555.tfrecord'

In [48]:
def parse_tfrecord_fn(example):
    feature_description = {
        'recipe_id': tf.io.FixedLenFeature([], tf.int64),
        # 'item_id': tf.io.FixedLenFeature([], tf.int64),
        # 'rating': tf.io.FixedLenFeature([], tf.float32),
        # 'image_filepath': tf.io.FixedLenFeature([], tf.string),
        # 'description': tf.io.FixedLenFeature([], tf.string),
    }
    example = tf.io.parse_single_example(example, feature_description)
    # example['image'] = tf.io.decode_jpeg(example['image_filepath'])
    return example

dataset = tf.data.TFRecordDataset(["item_55555.tfrecord"])

In [49]:
for el in dataset.take(1):
    print(el)
    print(parse_tfrecord_fn(el))

tf.Tensor(b'\n\x16\n\x14\n\trecipe_id\x12\x07\x1a\x05\n\x03\xb4\xc9\r', shape=(), dtype=string)
{'recipe_id': <tf.Tensor: shape=(), dtype=int64, numpy=222388>}
